# 实验2 电阻的温度特性
> Copyright(c) 2022 GeorgeDong32. All Rights Reserved.

## 测量铜电阻、NTC、PTC的温度特性

In [ ]:
from cProfile import label
import scipy.constants as cst 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
# plt防止中文乱码
plt.rcParams['font.sans-serif']=['SimHei']
plt.rcParams['axes.unicode_minus'] = False

In [ ]:
#拟合函数定义
def RCu(t,a,R0):
    return (1+a)*t*R0

def lnRNTC(T,A,B):
    return B/T + np.log(A)

def lnRPTC(T,A,B):
    return np.log(A) + B * T

In [ ]:
#读取数据
dCu = pd.read_csv("Cu.csv")
dNTC = pd.read_csv("NTC.csv")
dPTC = pd.read_csv("PTC.scv")

tCu = dCu.iloc[:,0]
rCu = (dCu.iloc[:,1] + dCu.iloc[:,2]) / 2

TNTC = dNTC.iloc[:,0] + 273
rNTC = (dNTC.iloc[:,1] + dNTC.iloc[:,2]) / 2
lnrNTC = np.log(rNTC)

TPTC = dPTC.iloc[:,0]
rPTC = (dPTC.iloc[:,1] + dPTC.iloc[:,2]) / 2
lnrPTC = np.log(rPTC)

popt1,pcov1=curve_fit(RCu,tCu,rCu)
rCu_fit = RCu(tCu,popt1[0],popt1[1])

popt2,pcov2=curve_fit(lnRNTC,TNTC,lnrNTC)
lnrNTC_fit = lnRNTC(TNTC,popt2[0],popt2[1])

popt3,pcov3=curve_fit(lnRPTC,TPTC,lnrPTC)
lnrPTC_fit = lnRPTC(TPTC,popt3[0],popt3[1])

### 铜电阻图像

In [ ]:
plt.figure(dpi=150)
plt.scatter(tCu,rCu,label="铜电阻测量值")
plt.plot(tCu,rCu_fit,label="铜电阻拟合曲线")
plt.title("铜电阻温度特性曲线")
plt.xlabel("\bft/°C")
plt.ylabel("$R_{Cu}/\omega$")
plt.annotate("$R_{Cu}$ = (1 + %.3f)t * %.3f" %(popt1[0],popt1[1]), xy=(30,55))
plt.legend()

### NTC电阻图像

In [ ]:
plt.figure(dpi=150)
plt.scatter(TNTC,lnrNTC,label="NTC电阻测量值")
plt.plot(TNTC,lnrNTC_fit,label="NTC电阻拟合曲线")
plt.title("NTC电阻温度特性曲线")
plt.xlabel("\bfT/K")
plt.ylabel("$lnR_{NTC}/ln\omega$")
plt.annotate("$lnR_{NTC}$ = (1 + %.3f)t * %.3f" %(popt2[0],popt2[1]), xy=(303,8))
plt.legend()

### PTC电阻图像

In [ ]:
plt.figure(dpi=150)
plt.scatter(TPTC,lnrPTC,label="PTC电阻测量值")
plt.plot(TPTC,lnrPTC_fit,label="PTC电阻拟合曲线")
plt.title("PTC电阻温度特性曲线")
plt.xlabel("\bfT/K")
plt.ylabel("$lnR_{PTC}/ln\omega$")
plt.annotate("$lnR_{PTC}$ = (1 + %.3f)t * %.3f" %(popt3[0],popt3[1]), xy=(303,5.8))
plt.legend()